### Import Required Package

In [24]:
import pandas as pd
import talib
import numpy as np
import json

### Import Tidal

In [25]:
import tidal as td

### Initialize Plumber

In [26]:
from pathlib import Path
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

PLUMBER_HOST = "https://dev-api.ddt-dst.cc/api/plumber/"
with open(f'{str(Path.home())}/.config/gcloud/application_default_credentials.json') as plumber_token:
    token = json.load(plumber_token)

### Load market data and compute technical indicators

In [27]:
import os
import pandas as pd

os.chdir('/home/jovyan/individualized-indicator')
# 定義你想要讀取檔案的目錄
directory = './emb'

start_date = "2022-06-01"
end_date = "2024-09-30"

# 初始化一個空的字典來存儲所有讀取的 DataFrame
dataframes = {}

# 遍歷目錄中的所有檔案
for filename in os.listdir(directory):
    if filename.endswith('.pkl'):
        # 組合完整的檔案路徑
        file_path = os.path.join(directory, filename)
        # 讀取 .pkl 檔案
        df = pd.read_pickle(file_path)
        
        # 使用檔案名稱（不包括副檔名）作為字典的鍵
        variable_name = filename.split('_')[2]
        
        # 將 DataFrame 儲存到字典中
        dataframes[variable_name] = df

keys_list = list(dataframes.keys())
# 將字符串轉換為 datetime 格式，然後進行排序
sorted_dates = sorted(pd.to_datetime(keys_list))
keys_list = [date.strftime('%Y-%m-%d') for date in sorted_dates]

total_df = pd.DataFrame()

for j in range(0,len(keys_list)):
    df_name = keys_list[j]
    selected_df = dataframes[df_name]    
    selected_df = selected_df.reset_index()
    stock_list = pd.DataFrame(selected_df.iloc[:,0].unique())
    total_df = pd.concat([total_df, stock_list], ignore_index=True)
    
unique_values_list = list(total_df.iloc[:,0].unique())
sorted_list = sorted(unique_values_list)

total_data = pd.read_parquet(
    f"{PLUMBER_HOST}stocks/tw/ohlcv",
    storage_options={
        "gcp-token": json.dumps(token),
        "start-date": start_date,
        "end-date": end_date,
        "tickers": ",".join([stock for stock in sorted_list]),
    },
)

total_data.index = total_data.index.set_levels(
    pd.to_datetime(total_data.index.levels[1]),level=1)

total_data.rename_axis(index={
    'ticker': 'instrument'
}, inplace=True)

total_data['new'] = -1
total_data.reset_index(inplace=True)

def filter_by_year_month_9(filter_data, select_year, select_mon):
    # 計算當前篩選年份和下一個年份
    next_year = select_year + 1
    
    # 構建篩選條件：當前年12月、次年1月、次年2月
    filter_data = filter_data[
        ((filter_data['datetime'].dt.year == next_year) & (filter_data['datetime'].dt.month == 1)) |
        ((filter_data['datetime'].dt.year == next_year) & (filter_data['datetime'].dt.month == 2)) |
        ((filter_data['datetime'].dt.year == next_year) & (filter_data['datetime'].dt.month == 3))
    ]
    
    return filter_data

def filter_by_year_month_12(filter_data, select_year, select_mon):
    # 計算當前篩選年份和下一個年份
    next_year = select_year + 1
    
    # 構建篩選條件：當前年12月、次年1月、次年2月
    filter_data = filter_data[
        ((filter_data['datetime'].dt.year == next_year) & (filter_data['datetime'].dt.month == 4)) |
        ((filter_data['datetime'].dt.year == next_year) & (filter_data['datetime'].dt.month == 5)) |
        ((filter_data['datetime'].dt.year == next_year) & (filter_data['datetime'].dt.month == 6))
    ]
    
    return filter_data

concat_df = pd.DataFrame()

# for i in range(0,len(keys_list)-1):
for i in range(0,len(keys_list)):
    df_name = keys_list[i]
    selected_df = dataframes[df_name]
    selected_df.reset_index(inplace=True)
    stock_df = selected_df.iloc[:,0]
    stock_df_unique = list(stock_df.unique())
    result = list(set(sorted_list) - set(stock_df_unique))
    filtered_df = total_data[total_data['instrument'].isin(result)]
    filtered_df = filtered_df.rename(columns={'volume': 'vol'})
    selected_df = pd.concat([selected_df, filtered_df], ignore_index=True)
    
    select_year = int(df_name.split('-')[0])
    select_mon = int(df_name.split('-')[1])
    select_mons = [(select_mon+4)%12, (select_mon+5)%12, (select_mon+6)%12]
    select_mons = [12 if mon == 0 else mon for mon in select_mons]
    
    selected_df['datetime'] = pd.to_datetime(selected_df['datetime'])
    # 過濾出所有6月的交易資料
    filter_data = selected_df[selected_df['datetime'].dt.month.isin(select_mons)]

    # 使用時，根據 select_year 和 select_mon 動態篩選數據
    
    if select_mon == 9:
        filter_data = filter_by_year_month_9(filter_data, select_year, select_mon)
    
    elif select_mon == 12:
        filter_data = filter_by_year_month_12(filter_data, select_year, select_mon) 

    else:
        filter_data = filter_data[filter_data['datetime'].dt.year == select_year]
    
    concat_df = pd.concat([concat_df, filter_data], ignore_index=True)
    
# i = len(keys_list)-1
# df_name = keys_list[i]
# selected_df = dataframes[df_name]
# selected_df.reset_index(inplace=True)
# stock_df = selected_df.iloc[:,0]
# stock_df_unique = list(stock_df.unique())
# result = list(set(sorted_list) - set(stock_df_unique))
# filtered_df = total_data[total_data['instrument'].isin(result)]
# filtered_df = filtered_df.rename(columns={'volume': 'vol'})
# selected_df = pd.concat([selected_df, filtered_df], ignore_index=True)
# concat_df = pd.concat([concat_df, selected_df], ignore_index=True)

concat_df.sort_values(by=['instrument', 'datetime'], inplace=True)
concat_df.reset_index(drop=True, inplace=True)
row = np.where(concat_df.iloc[:,0] == '6251')[0]
concat_df = concat_df.drop(index=row)
concat_df = concat_df.reset_index(drop=True)
row = np.where(concat_df.iloc[:,0] == '8406')[0]
concat_df = concat_df.drop(index=row)
concat_df = concat_df.reset_index(drop=True)
row = np.where(concat_df.iloc[:,0] == '6548')[0]
concat_df = concat_df.drop(index=row)
concat_df = concat_df.reset_index(drop=True)
quote_data = concat_df.set_index(['instrument', 'datetime'])
quote_data.columns.values[4] = 'volume'

/tmp/ipykernel_48149/1253896345.py:110: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  selected_df['datetime'] = pd.to_datetime(selected_df['datetime'])
/tmp/ipykernel_48149/1253896345.py:110: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  selected_df['datetime'] = pd.to_datetime(selected_df['datetime'])
/tmp/ipykernel_48149/1253896345.py:110: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  selecte

In [28]:
# i = len(keys_list)-1
# df_name = keys_list[i]
# selected_df = dataframes[df_name]
# selected_df.reset_index(inplace=True)
# stock_df = selected_df.iloc[:,0]
# stock_df_unique = list(stock_df.unique())
# result = list(set(sorted_list) - set(stock_df_unique))
# filtered_df = total_data[total_data['instrument'].isin(result)]
# filtered_df = filtered_df.rename(columns={'volume': 'vol'})
# selected_df = pd.concat([selected_df, filtered_df], ignore_index=True)
# concat_df = pd.concat([concat_df, selected_df], ignore_index=True)

### Strategy Lake API

In [29]:
data_api_url = ("https://dev-api.ddt-dst.cc/api/plumber/nextron/industry_prod_df")
df = pd.read_parquet(
    data_api_url,
    storage_options={
        "gcp-token": json.dumps(token),
    }
)

### Load benchmark data

In [30]:
benchmark_inst = "0050"
benchmark_data = pd.read_parquet(
    f"{PLUMBER_HOST}stocks/tw/ohlcv",
    storage_options={
        "gcp-token": json.dumps(token),
        "start-date": start_date,
        "end-date": end_date,
        "tickers": benchmark_inst,
    },
)
benchmark_data.index.set_levels(
    pd.to_datetime(benchmark_data.index.levels[1]),
    level=1,
    inplace=True,
)
benchmark_data.rename_axis(index={
    'ticker': 'instrument'
}, inplace=True)

benchmark_data

/tmp/ipykernel_48149/2173060872.py:11: FutureWarning: inplace is deprecated and will be removed in a future version.
  benchmark_data.index.set_levels(


open      high       low     close    volume
instrument datetime                                                    
0050       2022-06-01  119.2856  119.9775  119.0089  119.1473   6730117
           2022-06-02  118.5015  118.5015  117.7173  117.9018   4721923
           2022-06-06  118.3170  119.0089  117.5789  118.7321   4292361
           2022-06-07  118.0402  118.0402  117.0254  117.5328   5160242
           2022-06-08  118.5476  119.1011  118.2247  118.8244   4110313
...                         ...       ...       ...       ...       ...
           2024-09-24  183.2000  185.0000  182.1000  185.0000  10476465
           2024-09-25  187.7000  188.0000  187.3500  187.7500  12464313
           2024-09-26  189.6000  190.0000  188.7500  189.3000  17369543
           2024-09-27  190.2000  191.2500  188.6000  188.9000  11999510
           2024-09-30  186.5500  187.2000  183.9500  183.9500  14461065

[569 rows x 5 columns]

### Initialize Tidal
1. Initialize Tidal object
2. Add Quote data (pd.DataFrame)
3. Set strategy object (td.BaseStrategy)
4. Add metric objects (td.BaseMetic)

In [31]:
# Initialize Tidal object
tidal = td.Tidal(init_cash=10000000, slip_ticks=1, stock_config=td.StockConfig.TW, load_configs=True, reqMem="1000Mi")

# Add quote data
tidal.add_quote(quote_data)

# Set strategy object
tidal.set_strategy(td.strategy.TopkDropout(10, 1, 'new'))

# Set metric objects
tidal.add_metric(td.metric.AccountInfo())
tidal.add_metric(td.metric.AdditionalInfo())
tidal.add_metric(td.metric.PositionInfo())
tidal.add_metric(td.metric.Portfolio(benchmark_data.loc[benchmark_inst]))

### Stock Config

In [32]:
tidal.exchange.stock_config

InstConfig {Margin:0.0, Tick Size:0.002, Tick Value:0.002, Trade Unit:1000, Commission:0.0004275, Min Commission:20.0, Transaction Tax:0.003}

### Instrument Configs

In [33]:
tidal.exchange.inst_configs

{'MTX': InstConfig {Margin:46000.0, Tick Size:1.0, Tick Value:50.0, Trade Unit:1, Commission:20.0, Min Commission:0.0, Transaction Tax:2e-05},
 'TX': InstConfig {Margin:184000.0, Tick Size:1.0, Tick Value:200.0, Trade Unit:1, Commission:40.0, Min Commission:0.0, Transaction Tax:2e-05},
 'NQ': InstConfig {Margin:16500.0, Tick Size:0.25, Tick Value:5.0, Trade Unit:1, Commission:1.85, Min Commission:0.0, Transaction Tax:0.0}}

### Config Modification

In [34]:
# Set commission to 77% off
tidal.exchange.set_stock_config(commission=0.001425 * 0.23)
tidal.exchange.stock_config

InstConfig {Margin:0.0, Tick Size:0.002, Tick Value:0.002, Trade Unit:1000, Commission:0.00032775, Min Commission:20.0, Transaction Tax:0.003}

### Add New Instrument Config

In [35]:
# Add MGC config
tidal.exchange.set_config(instrument='MGC', margin=787., tick_size=0.1, tick_value=1., trade_unit=1, commission=2., min_commission=0., transaction_tax=0.)
tidal.exchange.inst_configs

{'MTX': InstConfig {Margin:46000.0, Tick Size:1.0, Tick Value:50.0, Trade Unit:1, Commission:20.0, Min Commission:0.0, Transaction Tax:2e-05},
 'TX': InstConfig {Margin:184000.0, Tick Size:1.0, Tick Value:200.0, Trade Unit:1, Commission:40.0, Min Commission:0.0, Transaction Tax:2e-05},
 'NQ': InstConfig {Margin:16500.0, Tick Size:0.25, Tick Value:5.0, Trade Unit:1, Commission:1.85, Min Commission:0.0, Transaction Tax:0.0},
 'MGC': InstConfig {Margin:787.0, Tick Size:0.1, Tick Value:1.0, Trade Unit:1, Commission:2.0, Min Commission:0.0, Transaction Tax:0.0}}

### Start Backtesting

In [36]:
tidal.backtest()

Tidal Backtesting: 100%|██████████| 483/483 [01:09<00:00,  6.97it/s, cash=6.66e+4, pnl=2.32e+5, position_cost=1.5e+7, value=1.53e+7]  


### Metric - AccountInfo

In [37]:
account_info = tidal.metrics["AccountInfo"].report
print(account_info)

                    cash    trade_cost  position_cost     slip_cost  \
datetime                                                              
2022-10-03  1.000000e+07  0.000000e+00   0.000000e+00  0.000000e+00   
2022-10-04  4.430399e+06  1.824839e+03   5.567776e+06  8.503089e+04   
2022-10-05  2.376126e+06  5.956308e+03   7.617338e+06  1.002646e+05   
2022-10-06  2.880454e+05  1.006048e+04   9.685790e+06  1.077339e+05   
2022-10-07  9.851374e+04  1.371435e+04   9.848086e+06  1.044285e+05   
...                  ...           ...            ...           ...   
2024-09-24  5.943588e+04  1.531893e+06   1.499209e+07  1.618090e+06   
2024-09-25  4.078134e+04  1.535338e+06   1.501323e+07  1.628899e+06   
2024-09-26  6.698297e+04  1.538886e+06   1.500032e+07  1.625599e+06   
2024-09-27  6.661549e+04  1.542297e+06   1.498708e+07  1.622406e+06   
2024-09-30  6.661549e+04  1.542297e+06   1.498708e+07  1.622406e+06   

                    pnl         value  max_drawdown      turnover  
datetime

### Metric - PositionInfo

In [38]:
position_df = tidal.metrics["PositionInfo"].report
print(position_df)

                       quantity      price  commission  slip_cost         pnl  \
instrument datetime                                                             
1101       2023-08-02   39000.0  36.476307  466.249275   947.9730 -14196.2730   
           2023-08-04   39000.0  36.476307  466.249275   947.9730  -8517.8730   
           2023-08-07   39000.0  36.476307  466.249275   947.9730   2838.9270   
           2023-08-22   30000.0  34.510500  339.324491  4368.0000  -4368.0000   
           2023-08-23   30000.0  34.510500  339.324491  4368.0000  -7281.0000   
...                         ...        ...         ...        ...         ...   
9917       2023-04-07   13000.0  97.826462  416.814100  8524.9112 -26485.7112   
           2023-04-10   13000.0  97.826462  416.814100  8524.9112 -20499.2112   
           2023-04-11   13000.0  97.826462  416.814100  8524.9112  -8524.9112   
           2023-04-12   13000.0  97.826462  416.814100  8524.9112   9435.8888   
           2023-04-13   1300

In [39]:
pi_report = tidal.metrics['PositionInfo'].report
pi_report.iloc[pi_report.index.get_level_values('datetime') == '2022-06-14']

,,quantity,price,commission,slip_cost,pnl,now_price
instrument,datetime,,,,,,


### Metric - AdditionalInfo

In [40]:
tidal.metrics['AdditionalInfo'].report

new
instrument datetime            
1101       2022-10-03  0.000665
           2022-10-04  0.000618
           2022-10-05  0.000491
           2022-10-06  0.000574
           2022-10-07  0.000857
...                         ...
9958       2024-06-24  0.000682
           2024-06-25  0.000826
           2024-06-26  0.001590
           2024-06-27  0.000963
           2024-06-28  0.000365

[281669 rows x 1 columns]

### Strategy Lake Submit

In [41]:
group = dict(zip(df["coid"].astype(str), df["tejind4_c"]))
submit_lake_backtest_result = tidal.submit_lake_backtest(account_info=account_info, position_df=position_df, benchmark_info=benchmark_data, group=group,lake_env="dev",strategy_id=19)
print(submit_lake_backtest_result)

{'strategy_id': 19, 'position_date': '2024-10-16', 'position_data': [{'asset': '1229', 'buy_date': '2024-07-02', 'price': 61.74, 'quantity': 32000.0, 'now_price': 61.7}, {'asset': '1301', 'buy_date': '2024-07-03', 'price': 55.18, 'quantity': 19000.0, 'now_price': 53.5}, {'asset': '1305', 'buy_date': '2024-07-04', 'price': 16.22, 'quantity': 92000.0, 'now_price': 17.8}, {'asset': '1326', 'buy_date': '2024-09-27', 'price': 42.28, 'quantity': 22000.0, 'now_price': 43.45}, {'asset': '3227', 'buy_date': '2024-06-20', 'price': 163.23, 'quantity': 6000.0, 'now_price': nan}, {'asset': '6170', 'buy_date': '2024-06-04', 'price': 55.26, 'quantity': 24000.0, 'now_price': nan}, {'asset': '6189', 'buy_date': '2024-06-27', 'price': 87.75, 'quantity': 19000.0, 'now_price': nan}, {'asset': '6284', 'buy_date': '2024-06-28', 'price': 89.65, 'quantity': 19000.0, 'now_price': nan}, {'asset': '6585', 'buy_date': '2024-05-24', 'price': 210.86, 'quantity': 9000.0, 'now_price': nan}, {'asset': '8112', 'buy_dat

### Trade Report

In [42]:
tidal.trade_report

,win_num,lose_num,trade_num,pos_num,win_rate,profit,loss,trade_cost,pnl
instrument,,,,,,,,,
6245,2.0,1.0,3.0,0.0,0.666667,650831.362989,-79804.129958,1.298778e+04,571027.233030
6577,1.0,0.0,1.0,0.0,1.000000,565719.322189,0.000000,5.781286e+03,565719.322189
5508,3.0,1.0,4.0,0.0,0.750000,500761.952516,-15081.363072,1.905895e+04,485680.589443
6561,2.0,0.0,2.0,0.0,1.000000,459621.831765,0.000000,1.148869e+04,459621.831765
2916,2.0,2.0,4.0,0.0,0.500000,435083.011379,-32573.360862,2.342881e+04,402509.650516
...,...,...,...,...,...,...,...,...,...
1326,6.0,16.0,22.0,1.0,0.272727,149573.761943,-318165.546393,1.013509e+06,-168591.784451
6570,0.0,1.0,1.0,0.0,0.000000,0.000000,-227259.155444,5.329782e+03,-227259.155444
8091,3.0,7.0,10.0,0.0,0.300000,54610.093748,-309095.219002,4.505402e+04,-254485.125254


### Traded instruments

In [43]:
tidal.account.trades.keys()

dict_keys(['2640', '1531', '8433', '3551', '1817', '3617', '5230', '3669', '8928', '1580', '6245', '4971', '5284', '4549', '4129', '4175', '5222', '8279', '4506', '6561', '9917', '6180', '2812', '6803', '2834', '5903', '5469', '3338', '4938', '4541', '8027', '8091', '3580', '3556', '6667', '6568', '2388', '6470', '6664', '2404', '2233', '6147', '6591', '5443', '6143', '1101', '6523', '8049', '3416', '2548', '2542', '4164', '2439', '3038', '6139', '4557', '6570', '8420', '3029', '6670', '6747', '2609', '5388', '2890', '6416', '8215', '1560', '6788', '2755', '8048', '6776', '6691', '2916', '2453', '5511', '3356', '6577', '5508', '6538', '4137', '4138', '3227', '6585', '6284', '6170', '8112', '5236', '8081', '3213', '6189', '1522', '5287', '1229', '1301', '1305', '1308', '1326'])

### Trading History

In [44]:
# tidal.account.trades['3015']

### Plot chart by using Plotly

In [45]:
# tidal.analyzer.inst_chart(instrument='3015', metric_name='AdditionalInfo', plot_type=td.PlotType.LINE, scale=1.0)

### Tidal Dashboard

In [46]:
tidal.tdboard()

 * Serving Flask app 'tidal.tdboard'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:42715
 * Running on http://10.136.1.88:42715
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:10.0.10.82 - - [16/Oct/2024 15:29:51] "GET / HTTP/1.1" 200 -
INFO:werkzeug:10.0.10.82 - - [16/Oct/2024 15:29:51] "GET /static/js/main.d754b0a3.js HTTP/1.1" 200 -
INFO:werkzeug:10.0.10.82 - - [16/Oct/2024 15:29:52] "GET /static/css/main.bf4d504b.css HTTP/1.1" 200 -
INFO:werkzeug:10.0.10.82 - - [16/Oct/2024 15:29:52] "GET /images/Tidal_Logo_white.png HTTP/1.1" 200 -
INFO:werkzeug:10.0.10.82 - - [16/Oct/2024 15:29:53] "GET /api/quote/inst_list HTTP/1.1" 200 -
INFO:werkzeug:10.0.10.82 - - [16/Oct/2024 15:29:53] "GET /api/trade/trade_report HTTP/1.1" 200 -
INFO:werkzeug:10.0.10.82 - - [16/Oct/2024 15:29:53] "GET /api/metric/metric_list HTTP/1.1" 200 -
INFO:werkzeug:10.0.10.82 - - [16/Oct/2024 15:29:5

In [ ]:
tidal.account.position_history